In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi
from datetime import datetime
import scipy
import itertools

Data on conflict events from ACLED

In [90]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\1997-01-01-2023-07-18-Eastern_Africa-Somalia.csv"
df = pd.read_csv(file)

#df=df[df['event_type'] != 'Protests']
#df=df[df['event_type'] != 'Strategic developments']
len(df)

41397

In [91]:
# Modify event_date column to datetime

v = df["event_date"].values
dt = [datetime.strptime(v[i], "%d %B %Y") for i in range(len(v))]
q=[]

for i in range(len(dt)):
    q.append(datetime.timestamp(dt[i]))
    
df.insert(loc=3, column='date_timestamp', value=q)
df = df.sort_values("date_timestamp")

df['event_date'] = pd.to_datetime(df['event_date'])
df=df.set_index('event_date') 

In [92]:
conflict=df.groupby([pd.Grouper(freq='40320min'),"admin1"]).count()
conflict.reset_index(level=[0, 1], inplace=True)
conflict=conflict[['event_date','admin1','year']].rename(columns={'year': 'conflicts','event_date': 'time'})

conflict['datetime_column'] = pd.to_datetime(conflict['time'])

# Aggregate the datetime objects by month
conf = conflict.groupby([pd.Grouper(key='time', freq='M'),'admin1'])['conflicts'].sum().to_frame()

In [93]:
dates = conf.index.get_level_values('time').unique()
districts = conf.index.get_level_values('admin1').unique()
all_combinations = pd.MultiIndex.from_product([dates, districts], names=['time', 'admin1'])

# Reindex the DataFrame with all combinations
conf = conf.reindex(all_combinations, fill_value=0).reset_index()

In [94]:
conflicts = conf.sort_values(by=['time', 'admin1'], ascending=[True, True])
conflicts.reset_index(drop=True, inplace=True)

In [95]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\all codes\temp_pre_somalia.csv"
temp_pre = pd.read_csv(file)
#rename columns
temp_pre=temp_pre.rename(columns={'region': 'admin1'})

In [96]:
# Add Banadir region with tmx and pre as mean of the neighbouring regions

district1 = 'Lower Shabelle'  
district2 = 'Middle Shabelle'  

# Step 2: Calculate the mean tmx and pre for the neighboring districts
mean_t = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['tmx'].mean()
mean_p = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['pre'].mean()

new_data = pd.DataFrame({ 'admin1': 'Banadir', 'tmx': mean_t, 'pre': mean_p}).reset_index()

# Step 4: Append the new DataFrame to the original DataFrame
df3 = pd.concat([temp_pre, new_data])

temp_pre = df3.sort_values(by=['time', 'admin1'], ascending=[True, True]).reset_index(drop=True)

In [97]:
temp_pre['month'] = temp_pre['time'].str[5:7]
temp_pre['month_year'] = temp_pre['time'].str[:7]
conflicts['time'] = conflicts['time'].dt.strftime('%Y-%m').values


In [98]:
temp_pre = temp_pre[['month_year','admin1','tmx','pre']]
temp_pre = temp_pre.rename(columns={'month_year':'time'})

In [99]:
temp_pre.to_csv('temp_pre_01_22.csv', index=False)

In [100]:
# Select a subset of the dataframe, where there is conflict data

start='1997-01'
end='2022-12'

temp_pre_97_22 = temp_pre[(temp_pre['time'] >= start) & (temp_pre['time'] <= end)]

In [103]:
# Perform an outer merge on the month_year column, which includes all month and year combinations from both dataframes
df_97_22 = pd.merge(temp_pre_97_22, conflicts, on=['time','admin1'], how='outer')
df_97_22 = df_97_22.fillna(0)

df_97_22=df_97_22[df_97_22['tmx'] != 0]
#df_97_22.drop(['month_year'], axis=1, inplace=True)

In [104]:
df_97_22.to_csv('df_97_22.csv', index=False)